#### Задание
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф" и сохранить результаты в pandas dataframe. Либо загрузите из pickle-файла df_nashdomrf.pkl, если не получается скачать.
Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных:
1. Привести колонки датафрейма к одному типу данных. 
2. Проверить индекс, чтобы он соответствовал порядку строк. 
3. Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки (например, заменить медианой, 0, -99999, удалить наблюдения с пропусками).

4. Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.
5. Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. 
6. Рассчитать объемы вводимого жилья по регионам и по годам. 
7. Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах. 
8. Визуализировать разброс, связь и распределение этих показателей по регионам. 
9. Сделать выводы по каждому из расчетов и графиков

In [1]:
import requests
import pandas as pd
import pickle
import numpy as np


In [2]:
offset_ = 1 # сколько элементов спарсить
limit_ = 1000 # количество объектов в на странице
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/places?offset={offset_}&limit={limit_}&searchValue='
# url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
# зашел на сайт наш дом.рф нажал на на инструменты разработчика, на вкладке -сеть при нажатии на следующую страницу
# появилась данная ссылка, стандартно выводит по 10 объектов
# url =f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/erz/main/filter?offset={offset_}&limit={limit_}&sortField=devShortNm&sortType=asc'
# offset при переходе на следующую страницу увеличивается на 10, 10.20.30, а Limit на 10
res = requests.get(url)

objects_data = res.json()
objects_data.get('data').get('list')[0]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
res

<Response [200]>

In [4]:
with open('df_nashdomrf.pkl', 'rb') as f:
    df_cop = pickle.load(f)
df_cop.head()

FileNotFoundError: [Errno 2] No such file or directory: 'df_nashdomrf.pkl'

In [ ]:
pd.set_option('display.max_columns', None)
df_cop.head(3)

In [ ]:
#Удаляем ненужные столбцы
df=df_cop.drop(['hobjId',
                 'pdId',
                  'objLkFreePlanDesc',
                  'objSquareLiving',
                  'rpdNum',
                  'rpdPdfLink',
                  'rpdIssueDttm',
                  'objLkLatitude',
                  'objLkLongitude',
                  'objCompensFundFlg',
                  'objProblemFlg',
                  'loadDttm',
                  'photoRenderDTO',
                  'objGuarantyEscrowFlg',
                  'objectType',
                  'miniUrl',
                  'residentialBuildings',
                  'newBuildingId',
                  'objFloorCnt',
                  'objFlatSq',
                  'objNonlivElemCnt',
                  'isAvailableWantLiveHere',
                  'objInfrstrBicycleLaneFlg',
                  'objInfrstrPlaygrndCnt',
                  'objInfrstrTrashAreaCnt',
                  'objInfrstrObjPrkngCnt',
                  'objInfrstrNotObjPrkngCnt',
                  'objInfrstrRampFlg',
                  'objInfrstrCurbLoweringFlg',
                  'objElevatorWheelchairCnt',
                  'objElevatorCargoCnt',
                  'soldOutPerc',
                  'nonlivFirstFloor',
                  'objectTransportInfo',
                  'conclusion',
                  'objLkSaleDepEmail',
                  'objGreenHouseFlg',
                  'objEnergyEffShortDesc',
                  'infrastructureIndexValue',
                  'developer.devId',
                  'generalContractorNm',
                  'developer.devShortNm',
                  'developer.devFullCleanNm',
                  'developer.problObjCnt',
                  'developer.buildObjCnt',
                  'developer.comissObjCnt',
                  'developer.regRegionDesc',
                  'developer.devSite',
                  'developer.devEmail',
                  'developer.devOgrn',
                  'developer.devKpp',
                  'developer.devLegalAddr',
                  'developer.devFactAddr',
                  'developer.lastRpdId',
                  'developer.fundGuarantyFlg',
                  'developer.devOrgRegRegionCd',
                  'developer.devEmplMainFullNm',
                  'developer.developerGroupName',
                  'developer.orgForm.id',
                  'developer.orgForm.fullForm',
                  'developer.orgForm.shortForm',
                  'developer.companyGroupId',
                  'developer.objGuarantyEscrowFlg',
                  'developer.govFundFlg',
                  'objLivCeilingHeight',
                  'quartography.objLivElem1KCnt',
                  'quartography.objLivElem2KCnt',
                  'quartography.objLivElem3KCnt',
                  'quartography.objLivElem4KCnt',
                  'quartography.objLivElemCnt',
                  'quartography.objLivElemEntrCnt',
                  'quartography.objLivElemEntrFloorAvg',
                  'nameObj',
                  'transportDistIndex',
                  'transportDistIndexValue',
                  'metro.id',
                  'metro.name',
                  'metro.line',
                  'metro.color',
                  'metro.time',
                  'metro.isWalk',
                  'metro.colors',
                  'complexShortNm',
                  'airQualityIndexValue',
                  'greenAreaIndexValue',
                  'developer.bankruptStage.bankruptStageCd',
                  'developer.bankruptStage.bankruptStageDesc',
                  'developer.bankruptStage.bankruptLawUrl',
                  'developer.orgBankruptMsgDttm'], axis=1)
df=df.rename(columns={'objElemLivingCnt':'Kol_kvart',
                      'objReady100PercDt':'SdachaDoma',
                      'objElemParkingCnt':'Parking',
                      'objLkClassDesc':'ClassDom',
                      'objFlatCnt':'KolKvartir',
                      'objInfrstrSportGroundCnt':'SportPlochadka',
                      'objPriceAvg':'AvgPrice'})
df.head(3)

In [ ]:
df.info()

In [ ]:
df['DatVidachKlyuch'] = df['DatVidachKlyuch'].replace(np.nan, 0)
df['SportPlochadka'] = df['SportPlochadka'].replace(np.nan, 0)
df['LiftPassagir'] = df['LiftPassagir'].replace(np.nan, 0)
df['AvgPrice'] = df['AvgPrice'].replace(np.nan, 0)
df['TelZastr'] = df['TelZastr'].replace(np.nan, 0)


In [ ]:
df.info()

In [ ]:
df['TelZastr'].count()

In [ ]:

df = df.astype({'address':'string'})

In [ ]:
df.dtypes